In [1]:
import pandas as pd
import math as m
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.model_selection import train_test_split

from time import time
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


Bad key "“backend" on line 1 in
/Users/rfdanti/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.1/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import pmdarima as pm

"""To display up to 50 columns of dataset"""
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 8)

"""To display all outputs of each cell"""
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


np.random.seed(100)

"""To get a timing of each function decorated"""
def timing(f):
    """
    Decorator for timing functions
    Usage:
    @timing
    def function(&):
        pass
    """
    
    def wrapper(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print('function:%r took: %2.2f sec' % (f.__name__, end - start))
        return result
    return wrapper

class Trainer():
    def __init__(self, input_directory, df_): #input_directory = path to the folder containing df. 
        self.input_directory = input_directory
        self.df = df_
                
    @timing
    def read(self):
        """Read raw data"""
        self.df = pd.read_csv(self.input_directory + self.df)
    
    @timing
    def auto_arima(self, nb_store):
        #Date as index
        self.df = self.df[self.df["Open"] == 1]
        self.df = pd.DataFrame(self.df, columns=['Date','Store','Sales'])
        self.df.sort_values(['Store', 'Date'], ascending=[True, True], inplace=True)
        #Re_indexation 
        self.df = self.df.reset_index(drop='True')
        
        """put the date as index"""
        self.df["Date"] = pd.to_datetime(self.df["Date"].astype(str), format = '%Y/%m/%d')
        self.df = self.df.set_index("Date")
        
        self.subset = self.df[(self.df["Store"]==(nb_store))]
        self.subset = self.subset.drop(["Store"], axis = 1)

        # Split train and test set  
        self.train = self.subset.loc[:'2015-06-13', :]
        self.test = self.subset.loc['2015-06-14':, :]

        # Build Model
        self.model = pm.auto_arima(self.subset.Sales, start_p=1, start_q=1,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=8, max_q=3, # maximum p and q
                          m=1,              # frequency of series
                          d=None,           # let model determine 'd'
                          seasonal=False, # No Seasonality
                          start_P=0, D=0, trace=True,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True)

        # Forecast
        self.n_periods = len(self.test)
        self.fc, self.confint = self.model.predict(n_periods=self.n_periods, return_conf_int=True)

        self.fc_series = pd.DataFrame(self.fc)
        self.fc_series.columns = ['predicted_ARIMA']
        self.fc_series = self.fc_series.reset_index(drop=True)
        self.fc_series = self.fc_series.set_index(self.test.index)
        self.test.columns = ['Sales']

        # Validate
        self.errors = abs(self.fc_series['predicted_ARIMA'] - self.test['Sales'])
        self.mae = round(np.mean(self.errors), 2)
        self.rmse = round(m.sqrt((1/len(self.test))*sum(self.errors**2)), 2)

        self.result = pd.merge(self.test, self.fc_series ,how = 'left',left_index = True, right_index = True)
        self.result["error_ARIMA"] = abs(self.result['predicted_ARIMA'] - self.result['Sales'])

        return self.result

/Users/rfdanti/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/Users/rfdanti/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
project = Trainer("/Users/rfdanti/Documents/UoM/2_data_analytics/coursework/CODE/", "full_trainset_v2.csv")
project.read()

function:'read' took: 2.46 sec


In [4]:
result_351 = project.auto_arima(351)
result_351 = result_351.reset_index(drop=False)
result_351.head()

Fit ARIMA: order=(1, 0, 1); AIC=13434.473, BIC=13453.105, Fit time=0.212 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13667.225, BIC=13676.541, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13438.600, BIC=13452.574, Fit time=0.058 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13508.523, BIC=13522.497, Fit time=0.016 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13417.455, BIC=13440.745, Fit time=0.260 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13432.413, BIC=13451.045, Fit time=0.127 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13350.668, BIC=13378.616, Fit time=0.419 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13344.854, BIC=13382.118, Fit time=1.086 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13345.848, BIC=13378.454, Fit time=0.621 seconds
Fit ARIMA: order=(4, 0, 3); AIC=13334.007, BIC=13375.929, Fit time=0.704 seconds
Fit ARIMA: order=(4, 0, 2); AIC=13347.819, BIC=13385.083, Fit time=0.956 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13346.034, BIC=13378.640, Fit time=0.813 seconds
Fit ARIMA: order=(5, 0, 3); 

,Date,Sales,predicted_ARIMA,error_ARIMA
0,2015-06-15,9850,6146.212449,3703.787551
1,2015-06-16,7449,6084.918832,1364.081168
2,2015-06-17,6829,5333.623101,1495.376899
3,2015-06-18,6628,5391.318968,1236.681032
4,2015-06-19,6716,4914.692239,1801.307761


In [5]:
result_351.to_csv("AutoARIMA_store351_6weeks.csv", index=False)

In [6]:
project = Trainer("/Users/rfdanti/Documents/UoM/2_data_analytics/coursework/CODE/", "full_trainset_v2.csv")
project.read()
result_279 = project.auto_arima(279)
result_279 = result_279.reset_index(drop=False)
result_279.head()

function:'read' took: 2.26 sec
Fit ARIMA: order=(1, 0, 1); AIC=14344.704, BIC=14363.331, Fit time=0.146 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14635.887, BIC=14645.201, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=14343.355, BIC=14357.326, Fit time=0.012 seconds
Fit ARIMA: order=(0, 0, 1); AIC=14411.547, BIC=14425.517, Fit time=0.018 seconds
Fit ARIMA: order=(2, 0, 0); AIC=14344.541, BIC=14363.167, Fit time=0.114 seconds
Fit ARIMA: order=(2, 0, 1); AIC=14341.024, BIC=14364.308, Fit time=0.262 seconds
Fit ARIMA: order=(3, 0, 1); AIC=14339.540, BIC=14367.480, Fit time=0.284 seconds
Fit ARIMA: order=(3, 0, 0); AIC=14342.608, BIC=14365.891, Fit time=0.143 seconds
Fit ARIMA: order=(3, 0, 2); AIC=14282.794, BIC=14315.391, Fit time=0.518 seconds
Fit ARIMA: order=(4, 0, 3); AIC=14277.377, BIC=14319.288, Fit time=0.895 seconds
Fit ARIMA: order=(3, 0, 3); AIC=14284.701, BIC=14321.955, Fit time=0.663 seconds
Fit ARIMA: order=(5, 0, 3); AIC=14279.274, BIC=14325.842, Fit time=2.212 secon

,Date,Sales,predicted_ARIMA,error_ARIMA
0,2015-06-15,19655,9610.638394,10044.361606
1,2015-06-16,11692,8107.714972,3584.285028
2,2015-06-17,12551,6771.895568,5779.104432
3,2015-06-18,10696,6889.414904,3806.585096
4,2015-06-19,10873,7072.885907,3800.114093


In [7]:
result_279.to_csv("AutoARIMA_store279_6weeks.csv", index=False)

In [8]:
project = Trainer("/Users/rfdanti/Documents/UoM/2_data_analytics/coursework/CODE/", "full_trainset_v2.csv")
project.read()
result_733 = project.auto_arima(733)
result_733 = result_733.reset_index(drop=False)
result_733.head()

function:'read' took: 2.12 sec
Fit ARIMA: order=(1, 0, 1); AIC=16474.152, BIC=16493.544, Fit time=0.192 seconds
Fit ARIMA: order=(0, 0, 0); AIC=16837.285, BIC=16846.981, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=16501.866, BIC=16516.410, Fit time=0.102 seconds
Fit ARIMA: order=(0, 0, 1); AIC=16608.054, BIC=16622.598, Fit time=0.037 seconds
Fit ARIMA: order=(2, 0, 1); AIC=16476.124, BIC=16500.364, Fit time=0.249 seconds
Fit ARIMA: order=(1, 0, 2); AIC=16476.093, BIC=16500.333, Fit time=0.238 seconds
Fit ARIMA: order=(2, 0, 2); AIC=16469.791, BIC=16498.879, Fit time=0.509 seconds
Fit ARIMA: order=(3, 0, 2); AIC=16348.230, BIC=16382.166, Fit time=1.873 seconds
Fit ARIMA: order=(3, 0, 1); AIC=16471.369, BIC=16500.457, Fit time=0.317 seconds
Fit ARIMA: order=(3, 0, 3); AIC=16314.709, BIC=16353.493, Fit time=3.782 seconds
Fit ARIMA: order=(2, 0, 3); AIC=16452.207, BIC=16486.143, Fit time=0.471 seconds
Fit ARIMA: order=(4, 0, 3); AIC=16299.801, BIC=16343.433, Fit time=3.468 secon

,Date,Sales,predicted_ARIMA,error_ARIMA
0,2015-06-14,16452,14691.727506,1760.272494
1,2015-06-15,16710,15542.076783,1167.923217
2,2015-06-16,16007,15940.098747,66.901253
3,2015-06-17,15036,14927.357884,108.642116
4,2015-06-18,15269,14584.851384,684.148616


In [9]:
result_733.to_csv("AutoARIMA_store733_6weeks.csv", index=False)

In [10]:
trainset = pd.read_csv("full_trainset_v2.csv")

#Date as index
df = trainset[trainset["Open"] == 1]
df = pd.DataFrame(df, columns=['Date','Store','Sales'])
df.sort_values(['Store', 'Date'], ascending=[True, True], inplace=True)
df = df.reset_index(drop=True)
df = df.set_index('Date')

In [11]:
import pmdarima as pm
dt = time()
mae_list = []
rmse_list = []
for i in range(len(df["Store"].unique())):
    try:
        # Filter by Store
        subset = df[(df["Store"]==(i+1))]
        subset = subset.drop(["Store"], axis = 1)

        # Split train and test set  
        train = subset.loc[:'2015-06-13', :]
        test = subset.loc['2015-06-14':, :]

        # Build Model
        model = pm.auto_arima(subset.Sales, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=8, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
        
        # Forecast
        n_periods = len(test)
        fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
        
        fc_series = pd.DataFrame(fc)
        fc_series.columns = ['Sales']
        fc_series = fc_series.reset_index(drop=True)
        fc_series = fc_series.set_index(test.index)
        test.columns = ['Sales']

        # Validate
        errors = abs(fc_series['Sales'] - test['Sales'])
        mae = round(np.mean(errors), 2)
        rmse = round(m.sqrt((1/len(test))*sum(errors**2)), 2)

        mae_list.append(mae)
        rmse_list.append(rmse)
    
    except:
        mae_list.append(np.nan)
        rmse_list.append(np.nan)
        pass

duration = time() - dt
print(duration)
print(rmse_list)

Fit ARIMA: order=(1, 0, 1); AIC=12542.222, BIC=12560.865, Fit time=0.160 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13028.092, BIC=13037.413, Fit time=0.004 seconds
Fit ARIMA: order=(1, 0, 0); AIC=12540.651, BIC=12554.633, Fit time=0.059 seconds
Fit ARIMA: order=(0, 0, 1); AIC=12734.251, BIC=12748.233, Fit time=0.091 seconds
Fit ARIMA: order=(2, 0, 0); AIC=12542.063, BIC=12560.705, Fit time=0.159 seconds
Fit ARIMA: order=(2, 0, 1); AIC=12516.866, BIC=12540.169, Fit time=0.501 seconds
Fit ARIMA: order=(3, 0, 1); AIC=12505.614, BIC=12533.577, Fit time=0.334 seconds
Fit ARIMA: order=(3, 0, 0); AIC=12530.950, BIC=12554.253, Fit time=0.236 seconds
Fit ARIMA: order=(3, 0, 2); AIC=12493.037, BIC=12525.661, Fit time=0.524 seconds
Fit ARIMA: order=(4, 0, 3); AIC=12465.627, BIC=12507.572, Fit time=1.065 seconds
Fit ARIMA: order=(3, 0, 3); AIC=12499.144, BIC=12536.428, Fit time=0.658 seconds
Fit ARIMA: order=(5, 0, 3); AIC=12478.160, BIC=12524.766, Fit time=1.027 seconds
Fit ARIMA: order=(4, 0, 2); 

Fit ARIMA: order=(2, 0, 1); AIC=14002.158, BIC=14025.480, Fit time=0.198 seconds
Fit ARIMA: order=(1, 0, 2); AIC=14001.951, BIC=14025.273, Fit time=0.313 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13977.005, BIC=14004.991, Fit time=0.523 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13803.726, BIC=13836.377, Fit time=0.722 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13967.231, BIC=13995.218, Fit time=0.270 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13805.415, BIC=13842.730, Fit time=1.042 seconds
Fit ARIMA: order=(4, 0, 3); AIC=13788.816, BIC=13830.796, Fit time=1.679 seconds
Fit ARIMA: order=(5, 0, 3); AIC=13787.427, BIC=13834.071, Fit time=1.695 seconds
Fit ARIMA: order=(5, 0, 2); AIC=13794.004, BIC=13835.984, Fit time=1.402 seconds
Fit ARIMA: order=(4, 0, 2); AIC=13805.213, BIC=13842.528, Fit time=1.353 seconds
Fit ARIMA: order=(6, 0, 3); AIC=13763.838, BIC=13815.146, Fit time=2.003 seconds
Fit ARIMA: order=(6, 0, 2); AIC=13792.900, BIC=13839.544, Fit time=1.597 seconds
Fit ARIMA: order=(7, 0, 3); 

Fit ARIMA: order=(6, 0, 3); AIC=13482.491, BIC=13533.800, Fit time=1.665 seconds
Fit ARIMA: order=(6, 0, 2); AIC=13519.193, BIC=13565.837, Fit time=1.811 seconds
Fit ARIMA: order=(5, 0, 2); AIC=13520.395, BIC=13562.375, Fit time=0.995 seconds
Fit ARIMA: order=(7, 0, 3); AIC=13475.866, BIC=13531.839, Fit time=3.411 seconds
Fit ARIMA: order=(7, 0, 2); AIC=13475.188, BIC=13526.496, Fit time=2.964 seconds
Fit ARIMA: order=(6, 0, 1); AIC=13519.504, BIC=13561.484, Fit time=1.222 seconds
Fit ARIMA: order=(8, 0, 2); AIC=13495.196, BIC=13551.169, Fit time=2.456 seconds
Fit ARIMA: order=(7, 0, 1); AIC=13517.940, BIC=13564.584, Fit time=1.249 seconds
Total fit time: 20.238 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13857.958, BIC=13876.580, Fit time=0.144 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13943.683, BIC=13952.994, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13876.825, BIC=13890.791, Fit time=0.015 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13897.440, BIC=13911.406, Fit time=0.016 secon

Fit ARIMA: order=(7, 0, 3); AIC=12867.510, BIC=12923.329, Fit time=2.926 seconds
Total fit time: 10.130 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13536.185, BIC=13554.817, Fit time=0.226 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13713.630, BIC=13722.946, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13548.288, BIC=13562.262, Fit time=0.069 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13603.683, BIC=13617.657, Fit time=0.020 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13528.231, BIC=13551.521, Fit time=0.150 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13527.634, BIC=13546.266, Fit time=0.148 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13486.946, BIC=13514.895, Fit time=0.227 seconds
Fit ARIMA: order=(4, 0, 1); AIC=13457.300, BIC=13489.906, Fit time=0.396 seconds
Fit ARIMA: order=(4, 0, 0); AIC=13504.786, BIC=13532.734, Fit time=0.195 seconds
Fit ARIMA: order=(4, 0, 2); AIC=13393.150, BIC=13430.414, Fit time=0.673 seconds
Fit ARIMA: order=(5, 0, 3); AIC=13361.586, BIC=13408.166, Fit time=0.869 secon

Fit ARIMA: order=(1, 0, 1); AIC=13932.244, BIC=13950.876, Fit time=0.083 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14207.650, BIC=14216.966, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13933.212, BIC=13947.186, Fit time=0.016 seconds
Fit ARIMA: order=(0, 0, 1); AIC=14017.930, BIC=14031.904, Fit time=0.020 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13914.819, BIC=13938.109, Fit time=0.266 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13931.411, BIC=13950.043, Fit time=0.154 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13922.635, BIC=13950.583, Fit time=0.398 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13917.995, BIC=13950.601, Fit time=0.819 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13928.212, BIC=13956.160, Fit time=0.335 seconds
Total fit time: 2.101 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13268.564, BIC=13287.186, Fit time=0.084 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13616.036, BIC=13625.347, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13267.635, BIC=13281.601, Fit time=0.017 second

...

Fit ARIMA: order=(4, 0, 3); AIC=13793.383, BIC=13835.282, Fit time=2.894 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13786.578, BIC=13819.166, Fit time=0.417 seconds
Total fit time: 10.440 seconds
Fit ARIMA: order=(1, 0, 1); AIC=14310.438, BIC=14329.080, Fit time=0.124 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14499.026, BIC=14508.347, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=14315.452, BIC=14329.434, Fit time=0.017 seconds
Fit ARIMA: order=(0, 0, 1); AIC=14368.639, BIC=14382.620, Fit time=0.017 seconds
Fit ARIMA: order=(2, 0, 1); AIC=14308.410, BIC=14331.713, Fit time=0.116 seconds
Fit ARIMA: order=(2, 0, 0); AIC=14307.097, BIC=14325.739, Fit time=0.026 seconds
Fit ARIMA: order=(3, 0, 1); AIC=14269.615, BIC=14297.578, Fit time=0.412 seconds
Fit ARIMA: order=(4, 0, 1); AIC=14237.075, BIC=14269.699, Fit time=0.354 seconds
Fit ARIMA: order=(4, 0, 0); AIC=14269.161, BIC=14297.125, Fit time=0.073 seconds
Fit ARIMA: order=(4, 0, 2); AIC=14196.670, BIC=14233.954, Fit time=0.701 secon

Fit ARIMA: order=(2, 0, 3); AIC=13211.252, BIC=13243.903, Fit time=0.549 seconds
Fit ARIMA: order=(1, 0, 2); AIC=13269.090, BIC=13292.412, Fit time=0.265 seconds
Fit ARIMA: order=(1, 0, 3); AIC=13269.789, BIC=13297.776, Fit time=0.381 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13222.623, BIC=13259.939, Fit time=0.946 seconds
Total fit time: 3.924 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13656.147, BIC=13674.779, Fit time=0.117 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13707.999, BIC=13717.315, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13663.769, BIC=13677.743, Fit time=0.017 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13672.959, BIC=13686.933, Fit time=0.017 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13654.815, BIC=13678.105, Fit time=0.223 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13653.908, BIC=13672.541, Fit time=0.138 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13648.821, BIC=13676.769, Fit time=0.314 seconds
Fit ARIMA: order=(4, 0, 1); AIC=13599.307, BIC=13631.913, Fit time=0.358 second

Fit ARIMA: order=(3, 0, 1); AIC=13369.824, BIC=13397.795, Fit time=0.317 seconds
Fit ARIMA: order=(3, 0, 0); AIC=13368.792, BIC=13392.101, Fit time=0.136 seconds
Total fit time: 1.039 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13400.407, BIC=13419.039, Fit time=0.154 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13674.038, BIC=13683.354, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13422.174, BIC=13436.148, Fit time=0.075 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13519.690, BIC=13533.664, Fit time=0.030 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13375.044, BIC=13398.334, Fit time=0.224 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13392.056, BIC=13410.688, Fit time=0.114 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13363.872, BIC=13391.820, Fit time=0.413 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13346.581, BIC=13383.845, Fit time=0.799 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13361.496, BIC=13394.103, Fit time=0.523 seconds
Fit ARIMA: order=(4, 0, 3); AIC=13331.629, BIC=13373.551, Fit time=1.153 second

Fit ARIMA: order=(5, 0, 3); AIC=13029.237, BIC=13075.791, Fit time=1.231 seconds
Fit ARIMA: order=(5, 0, 2); AIC=13067.672, BIC=13109.571, Fit time=0.743 seconds
Fit ARIMA: order=(6, 0, 3); AIC=13030.738, BIC=13081.948, Fit time=1.801 seconds
Total fit time: 8.459 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13171.382, BIC=13190.025, Fit time=0.143 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13589.089, BIC=13598.410, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13171.510, BIC=13185.492, Fit time=0.061 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13310.190, BIC=13324.172, Fit time=0.059 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13173.167, BIC=13196.470, Fit time=0.240 seconds
Fit ARIMA: order=(1, 0, 2); AIC=13172.783, BIC=13196.086, Fit time=0.247 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13167.328, BIC=13195.291, Fit time=0.327 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13137.389, BIC=13170.013, Fit time=0.362 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13172.508, BIC=13200.471, Fit time=0.256 second

Fit ARIMA: order=(3, 0, 2); AIC=13387.336, BIC=13419.977, Fit time=0.690 seconds
Fit ARIMA: order=(4, 0, 3); AIC=13386.838, BIC=13428.806, Fit time=1.688 seconds
Fit ARIMA: order=(5, 0, 3); AIC=13354.982, BIC=13401.613, Fit time=1.380 seconds
Fit ARIMA: order=(5, 0, 2); AIC=13379.360, BIC=13421.329, Fit time=0.842 seconds
Fit ARIMA: order=(4, 0, 2); AIC=13401.703, BIC=13439.008, Fit time=0.449 seconds
Fit ARIMA: order=(6, 0, 3); AIC=13355.917, BIC=13407.212, Fit time=1.199 seconds
Total fit time: 8.008 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13748.433, BIC=13767.065, Fit time=0.082 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13772.748, BIC=13782.064, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13754.370, BIC=13768.344, Fit time=0.019 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13758.646, BIC=13772.620, Fit time=0.017 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13745.214, BIC=13768.504, Fit time=0.117 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13743.481, BIC=13762.113, Fit time=0.032 second

In [12]:
rmse_list = pd.DataFrame(rmse_list, columns=["RMSE"])
mae_list = pd.DataFrame(mae_list, columns=["MAE"])

store = list(range(1,1116))
store_series = pd.DataFrame(store, columns=["Store"])
result = pd.merge(store_series, rmse_list, how = 'left',left_index = True, right_index = True)
result = pd.merge(result, mae_list, how = 'left',left_index = True, right_index = True)
result.head()

,Store,RMSE,MAE
0,1,1052.76,880.66
1,2,1573.07,1287.98
2,3,2104.92,1787.02
3,4,2105.90,1643.34
4,5,1686.53,1341.39


In [13]:
result.RMSE.mean()

1808.650295964127

In [14]:
result.RMSE.median()

1716.03

In [15]:
result.to_csv("arima_result_6weeks.csv", index=False)